<img src="../docs/sa_logo.png" width="250" align="left">

# Image Classification with Amazon Rekognition and SuperAnnotate 


## Introduction

This tutorial shows an example of solving ```Image classification task``` with [SuperAnnotate](https://www.superannotate.com/) and [Amazon Rekognition](https://us-west-2.console.aws.amazon.com/rekognition/home?region=us-west-2#/).

The main goal of this tutorial is to show how one could annotate some part of data with ```SuperAnnotate``` tools and then build a model with ```Rekognition``` to automatically annotate the rest of data. These automatically generated annotations may be additionaly checked and modified manually.

All the experiments described in this tutorial were done with [RESISC45](https://paperswithcode.com/dataset/resisc45) dataset.

The tutorial starts with the assumption that we have partially annotated dataset of images.
The data is stored on S3 bucket and splitted into two parts: 
* train (~17%) $-$ annotated data for training
* unlabeled (~83%) $-$ data that will be annotated by the model

This folders are connected with existing SuperAnnotate project and train dataset has already been annotated manually. 

![](../docs/image_classification_rekognition/folders.png)

In the examples below we used ```SuperAnnotate SDK``` and ```Boto3 SDK```. Some parts of code used here are provided as examples in [SuperAnnotate](https://doc.superannotate.com/docs/getting-started) and [Boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) documentations.

In this tutorial we will go through the following steps:

$\textbf{1.}$ [Environmental setup](#envin_setup)

$\textbf{2.}$ [Create Rekognition project](#create_rekognition_project)


$\textbf{3.}$ [Create empty dataset](#create_empty_dataset)


$\textbf{4.}$ [Download Labels from SuperAnnotate](#download_labels_from_SA)


$\textbf{5.}$ [Upload labels to Rekognition Project for train data](#upload_labels_to_rekognition)


$\textbf{6.}$ [Move 20% of training data to test dataset](#move_20_to_test)


$\textbf{7.}$ [Train the model](#train_the_model)


$\textbf{8.}$ [Start the model](#start_the_model)


$\textbf{9.}$ [Test the prediction](#predict)


$\textbf{10.}$ [Predict unlabeled images](#predict_unlabeled_images)


$\textbf{11.}$ [Make SA annotations](#make_sa_annotations)


$\textbf{12.}$ [Upload new annotations to SA](#upload_new_annotations_to_sa)

In [2]:
! pip install superannotate==4.4.7 #SA SDK installation
! pip install boto3 # install boto3 client

### 1.1 User Variables Setup

In [21]:
#SuperAnnotate SDK token
SA_TOKEN = "ADD YOUR TOKEN"

In [22]:
SA_PROJECT_NAME = "ADD SUPERANNOTATE PROJECT NAME"

In [23]:
#name for Rekognition project we will create
REKOGNITION_PROJECT_NAME = "ADD REKOGNITION PROJECT NAME"

### 1.2 Constants Setup

In [19]:
import boto3
import os

SuperAnnotate Python SDK functions work within the team scope of the platform, so a team-level authorization is required.

To authorize the package in a given team scope, get the authorization token from the team settings page.

In [ ]:
sa_client = SAClient(token=SA_TOKEN) ## SuperAnnotate client

Data that is shown on SuperAnnotate page is actually stored on AWS S3 Bucket.
Here we provide name of this bucket.

In [ ]:
bucket_name = "ADD YOUR BUCKET NAME" # bucket where the data is stored

We should also create clients to be able to work with S3 and Rekognition

In [ ]:
s3_client = boto3.client('s3') ## S3 client

rek_client = boto3.client('rekognition') ## Rekognition client

Images shown on SuperAnnotate page are stored in S3 bucket.
We can add them to Rekognition project and train the model using them.

Before that we should get links to all of them.
Since S3 SDK could list only 1000 objects per step, we could do it iteratively.

In [ ]:
#this dict will contain S3 paths to train and unlabeled images
data_links_dict = {'train': [],
                   'unlabeled': []}

#path to the folder containing images in S3 bucket
BUCKET_FOLDER_PATH = '/path/to/data/'

image_format = '.jpg'
start_key = ''
max_keys = 1000

for subset_name in ['train', 'unlabeled']:
    while True:
        response = s3_client.list_objects_v2(Bucket=bucket_name,
                                             Prefix=f'{BUCKET_FOLDER_PATH}/{subset_name}/',
                                             StartAfter=start_key,
                                             MaxKeys=max_keys)
        objects = response['Contents']
        for obj in objects:
            path = obj['Key']
            if path.endswith(image_format):
                data_links_dict[subset_name].append(obj['Key'])
        start_key = objects[-1]['Key']
        if len(objects) < max_keys:
            start_key = ''
            break

## 2. Create Rekognition project
<a id='create_rekognition_project'></a>

Now we can move to Rekognition and create an empty project there.

In [ ]:
response = rek_client.create_project(ProjectName=REKOGNITION_PROJECT_NAME)
project_arn = response['ProjectArn'] ## store project's ARN to use it later
print(project_arn)

## 3. Create Empty Dataset
<a id='create_empty_dataset'></a>

In order to train the model on our data we should upload the data to Rekognition platform.
We create an empty dataset for our data.

In [ ]:
import boto3
import argparse
import time
from botocore.exceptions import ClientError
from datetime import datetime



def create_empty_dataset(rek_client, project_arn, dataset_type):
    """
    Creates an empty Amazon Rekognition Custom Labels dataset.
    :param rek_client: The Amazon Rekognition Custom Labels Boto3 client.
    :param project_arn: The ARN of the project in which you want to create a dataset.
    :param dataset_type: The type of the dataset that you want to create (train or test).
    """
    try:
        #Create the dataset
        print(f"Creating empty {dataset_type} dataset for project {project_arn}")

        dataset_type = dataset_type.upper()

        response = rek_client.create_dataset(ProjectArn=project_arn,
                                             DatasetType=dataset_type)

        dataset_arn = response['DatasetArn']

        print(f"dataset ARN: {dataset_arn}")

        finished = False
        while not finished:

            dataset = rek_client.describe_dataset(DatasetArn=dataset_arn)

            status = dataset['DatasetDescription']['Status']
            
            if status == "CREATE_IN_PROGRESS":
                
                print((f"Creating dataset: {dataset_arn} "))
                time.sleep(5)
                continue

            if status == "CREATE_COMPLETE":
                print(f"Dataset created: {dataset_arn}")
                finished = True
                continue

            if status == "CREATE_FAILED":
                raise Exception (f"Dataset creation failed: {status} : {dataset_arn}")
                
            
        return dataset_arn
       
    except ClientError as err:  
        print(f"Could not create dataset: {err.response['Error']['Message']}")
        raise

In [ ]:
dataset_type = 'train'

try:
    print(f"Creating empty {dataset_type} dataset for project {project_arn}")

    #Create the empty dataset
    train_dataset_arn=create_empty_dataset(rek_client, 
                                           project_arn,
                                           dataset_type.lower())

    print(f"Finished creating empty dataset: {train_dataset_arn}")

except Exception as err:
    print(f"Problem creating empty dataset: {err}")

## 4. Download Labels from SA
<a id='download_labels_from_SA'></a>

In [ ]:
filenames = [os.path.basename(x) for x in data_links_dict['train']]

In [ ]:
annotations = sa_client.get_annotations(project=f"{SA_PROJECT_NAME}/train", 
                                        items=filenames)

labels = [a['instances'][0]['className'] for a in annotations]

## 5. Upload labels to Rekognition Project for train data
<a id='upload_labels_to_rekognition'></a>

Now we can add our images from S3 bucket to Rekognition Dataset that was created in previous section.

We will do it via manifest file that will containt urls for all our images. 

For more information read about [adding images with manifest files](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/md-add-images.html)

In [ ]:
def create_manifest_file(bucket_name,
                         folder_path,
                         filenames,
                         labels,
                         manifest_file_name):
    s3_folder = f"s3://{bucket_name}/{folder_path}"
    image_count = 0
    with open(manifest_file_name, "w", encoding="UTF-8") as output_file:
        for filename, class_name in zip(filenames, labels):
            if len(filename) ==  0:
                continue
                
            json_line = {"source-ref": f"{s3_folder}/{filename}",
                         "imagelabel": 0,
                         "imagelabel-metadata": {"class-name": class_name,
                                                 "confidence": 1.0,
                                                 "human-annotated": "yes",
                                                 "type": "groundtruth/image-classification",
                                                 "creation-date": datetime.now().strftime("%Y-%m-%d:%H:%M:%S"),
                                                 "job-name": "Test job"}}
            output_file.write(json.dumps(json_line))
            output_file.write('\n')
            image_count += 1
    return image_count

In [ ]:
updates_file_name = './train.manifest'

create_manifest_file(bucket_name=data_bucket_name,
                     folder_path=BUCKET_FOLDER_PATH,
                     filenames=filenames,
                     labels=lables,
                     manifest_file_name=updates_file_name)

In [ ]:
def update_dataset_entries(rek_client, dataset_arn, updates_file):
    """
    Adds dataset entries to an Amazon Rekognition Custom Labels dataset.    
    :param rek_client: The Amazon Rekognition Custom Labels Boto3 client.
    :param dataset_arn: The ARN of the dataset that yuo want to update.
    :param updates_file: The manifest file of JSON Lines that contains the updates. 
    """

    try:
        status = ""
        status_message = ""

        #Update dataset entries
        print(f"Updating dataset {dataset_arn}")


        with open(updates_file) as f:
            manifest_file = f.read()

        
        changes=json.loads('{ "GroundTruth" : ' +
            json.dumps(manifest_file) + 
            '}')
        print(f"{len(changes['GroundTruth'])} to add")
        rek_client.update_dataset_entries(Changes=changes,
                                          DatasetArn=dataset_arn)
        print(f"Updated dataset {dataset_arn}")
        finished = False
        while finished == False:

            dataset = rek_client.describe_dataset(DatasetArn=dataset_arn)

            status = dataset['DatasetDescription']['Status']
            status_message = dataset['DatasetDescription']['StatusMessage']
            
            if status == "UPDATE_IN_PROGRESS":
                print((f"Updating dataset: {dataset_arn} "))
                time.sleep(5)
                continue

            if status == "UPDATE_COMPLETE":
                print(f"Dataset updated: {status} : {status_message} : {dataset_arn}")
                finished=True
                continue

            if status == "UPDATE_FAILED":
                print(f"Dataset update failed: {status} : {status_message} : {dataset_arn}")
                raise Exception (f"Dataset update failed: {status} : {status_message} : {dataset_arn}")
                

            print(f"Failed. Unexpected state for dataset update: {status} : {status_message} : {dataset_arn}")
            raise Exception(f"Failed. Unexpected state for dataset update: {status} : {status_message} :{dataset_arn}")
            
        print(f"Added entries to dataset")
        
        return status, status_message
   
    
    except ClientError as err:  
        print(f"Couldn't update dataset: {err.response['Error']['Message']}")
        raise

In [ ]:
try:
    print(f"Updating dataset {train_dataset_arn} with entries from {updates_file_name}.")

    status, status_message=update_dataset_entries(rek_client, 
                                                  train_dataset_arn,
                                                  updates_file_name)

    print(f"Finished updates dataset: {status} : {status_message}")
except Exception as err:
    print(f"Problem updating dataset: {err}")

## 6. Move 20% of training data to test dataset
<a id='move_20_to_test'></a>

To train the model we should specify training and test datasets.
We will move 20% of our training data from train to test dataset. 


After model training is complete the model's performance will be evaluated using test dataset.

In [ ]:
dataset_type = 'test'

try:
    print(f"Creating empty {dataset_type} dataset for project {project_arn}")

    #Create the empty dataset
    test_dataset_arn=create_empty_dataset(rek_client, 
                                          project_arn,
                                          dataset_type.lower())

    print(f"Finished creating empty dataset: {test_dataset_arn}")

except Exception as err:
    print(f"Problem creating empty dataset: {err}")

We can distribute data between test and train automatically by using $ \textit{distribute_dataset_entries}$ method from boto3.

In [ ]:
datasets = json.loads(
            '[{"Arn" : "' + str(train_dataset_arn) + '"},{"Arn" : "' + str(test_dataset_arn) + '"}]')

rek_client.distribute_dataset_entries(Datasets=datasets)

## 7. Train the model
<a id='train_the_model'></a>

Now we can train our model. We will use $create\_project\_version$ from boto3.

In [ ]:
def train_model(rek_client, 
                project_arn, 
                version_name, 
                output_bucket, 
                output_folder, 
                tag_key = None, 
                tag_key_value = None):
    """
    Trains an Amazon Rekognition Custom Labels model.
    :param rek_client: The Amazon Rekognition Custom Labels Boto3 client.
    :param project_arn: The ARN of the project in which you want to train a model.
    :param version_name: A version for the model.
    :param output_bucket: The S3 bucket that hosts training output.
    :param output_folder: The path for the training output within output_bucket
    :param tag_key: The name of a tag to attach to the model. Pass None to exclude
    :param tag_key_value: The value of the tag. Pass None to exclude

    """

    try:
        #Train the model

        status="" 
        print(f"training model version {version_name} for project {project_arn}")


        output_config = json.loads(
            '{"S3Bucket": "'
            + output_bucket
            + '", "S3KeyPrefix": "'
            + output_folder
            + '" }  '
        )

        tags = {}

        if tag_key != None and tag_key_value != None:
            tags = json.loads('{"' + tag_key + '":"' + tag_key_value + '"}')

        response = rek_client.create_project_version(ProjectArn=project_arn, 
                                                     VersionName=version_name,
                                                     OutputConfig=output_config,
                                                     Tags=tags)

        print(f"Started training: {response['ProjectVersionArn']}")

        # Wait for the project version training to complete
        project_version_training_completed_waiter = rek_client.get_waiter('project_version_training_completed')
        project_version_training_completed_waiter.wait(ProjectArn=project_arn,
                                                       VersionNames=[version_name])
    

        #Get the completion status
        describe_response = rek_client.describe_project_versions(ProjectArn=project_arn,
                                                                 VersionNames=[version_name])
        for model in describe_response['ProjectVersionDescriptions']:
            print("Status: " + model['Status'])
            print("Message: " + model['StatusMessage']) 
            status = model['Status']


        print(f"finished training")

        return response['ProjectVersionArn'], status
    
    except ClientError as err:  
        print(f"Couldn't create model: {err.response['Error']['Message']}")
        raise

We should specify name for model's version and place (bucket and folder) for Rekognition to store its output. 

In [ ]:
version_name = "NAME OF MODEL'S VERSION"
output_bucket = "OUTPUT BUCKET NAME"
output_folder = "/PATH/TO/OUTPUT"

In [ ]:
s3_client.put_object(Bucket=bucket_name, Key=(output_folder+'/'))

Here we will start model training

In [ ]:
try:

    print(f"Training model version {version_name} for project {project_arn}")

    model_arn, status = train_model(rek_client, 
                                    project_arn,
                                    version_name,
                                    output_bucket,
                                    output_folder)

    print(f"Finished training model: {model_arn}")
    print(f"Status: {status}")

except Exception as err:
    print(f"Problem training model: {err}")

While the code cell above is running you could see the model with status "TRAINING_IN_PROGRESS" on Rekognition webpage.

![](../docs/image_classification_rekognition/training_in_progress.png "Training in progress") 

## 8. Start the model

<a id='start_the_model'></a>

Once training process is completed we can see that model status on Rekognition page is now "Training completed" and model is ready to run.

![](../docs/image_classification_rekognition/model_ready_to_run.png)

Since we have our model trained we can now use it to get the predictions for unlabeled data

Before that we should start the model.

In [ ]:
def start_model(client, project_arn, model_arn, version_name, min_inference_units):

    try:
        # Start the modelstart_the_model
        print('Starting model: ' + model_arn)
        response=client.start_project_version(ProjectVersionArn=model_arn, MinInferenceUnits=min_inference_units)
        # Wait for the model to be in the running state
        project_version_running_waiter = client.get_waiter('project_version_running')
        project_version_running_waiter.wait(ProjectArn=project_arn, VersionNames=[version_name])

        #Get the running status
        describe_response=client.describe_project_versions(ProjectArn=project_arn,
            VersionNames=[version_name])
        for model in describe_response['ProjectVersionDescriptions']:
            print("Status: " + model['Status'])
            print("Message: " + model['StatusMessage']) 
    except Exception as e:
        print(e)
        
    print('Done...')

In [ ]:
start_model(rek_client,
            project_arn, 
            model_arn, 
            version_name, 
            min_inference_units=1)

After the cell above is finished we could see that model status on Rekognition page is now "Running"

![](../docs/image_classification_rekognition/model_is_running.png)

## 9. Prediction test
<a id='predict'></a>

Now the model is ready to run and we can get prediction for any unlabeled image we have.
We have to provide a path to S3 bucket and folder where the image is located.

In [ ]:
def show_custom_labels(client, model, bucket, photo, min_confidence):
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 
                                                                 'Name': photo}},
                                           MinConfidence=min_confidence,
                                           ProjectVersionArn=model)
    return response['CustomLabels']

In [ ]:
filename = 'FILE TO TEST'
photo = f'{BUCKET_FOLDER_PATH}/unlabeled/{filename}'

In [ ]:
min_confidence = 10 ## minimum confindence model should have for the label to detect it

labels = show_custom_labels(rek_client, model_arn, bucket_name, photo, min_confidence)
labels

## 10. Predict unlabeled images
<a id='predict_unlabeled_images'></a>

We can now get the predictions for all our unlabeled images.

We could use the function $\textit{show_custom_labels}$ and dictionary with S3 paths to all unlabeld images $data\_ links\_dict$ that we created above.

In [ ]:
predicted_labels  = {}

for photo_path in tqdm(data_links_dict['unlabeled']):
    labels = show_custom_labels(rek_client, model_arn, bucket, photo_path, min_confidence)
    predicted_labels[photo_path] = labels

## 11. Make SA annotations
<a id='make_sa_annotations'></a>

Based on predictions made by the model we should now create annotations in SuperAnnotate format to be able to upload them to SuperAnnotate.

In [ ]:
ANNOTATIONS_FOLDER = 'PATH/TO/LOCAL/DIR/' # local folder to store .json files with annotations
for image_path, label in predicted_labels.items():
    filename = os.path.basename(image_path)
    js_annotation = {"metadata": {"name": filename},
                     "instances": [{"type": "tag",
                                    "className": label[0]['Name']}]}
    with open(f'{ANNOTATIONS FOLDER}/{filename}.json','w') as f:
        json.dump(js_annotation, f)

## 12. Upload new annotations to SA 
<a id='upload_new_annotations_to_sa'></a> 

Now we could upload annotations generated on the previous step back to SuperAnnnotate.

In [ ]:
def read_js(filename):
    with open(filename) as f:
        js = json.load(f)
    return js 

In [ ]:
outputs = []
full_dirname = ANNOTATIONS_FOLDER
files = os.listdir(full_dirname)
files_per_step = 500
steps = len(files) // files_per_step + 1

for step in range(steps):
    start = step * files_per_step
    end = min((step + 1)* files_per_step, len(files))

    batch = [read_js(os.path.join(full_dirname, f)) for f in files[start: end]]

    outputs.append(sa_client.upload_annotations(project=f'{SA_PROJECT_NAME}/unlabeled/', annotations=batch))

Now we can look at unlabeled folder at the SuperAnnotate page and see the predictions made by our model.


All files in unlabeled folder changed their status.

![](../docs/image_classification_rekognition/unlabeled_status_changed.png)

We can open any of these files and check whether it is annitated correctly.

![](../docs/image_classification_rekognition/airplane_example.png)